In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
import pyclifford as pc
import torch

from numba import njit
import seaborn as sns
sns.set_theme()

import statistics

In [4]:
def one_layer_brick_wall(circ,even=True):
    for ii in range(int(circ.N//2)):
        if even:
            circ.gate(2*ii,2*ii+1)
        else:
            circ.gate((2*ii+1),(2*ii+2)%circ.N)
    return circ

In [5]:
def theta(N_QUBITS,DEPTH):
    """

    Arguments:
        number of qubits, depth
    
    Returns:
        Random 2-qubit Clifford gates for a single layer.
        
    """
    
    
    theta= np.random.randint(2, size=(DEPTH,N_QUBITS))
    
    return theta

In [6]:
def one_layer_measurement(circ,the,layer):
    """
    Input:
        
    """
     
    positions=[]
    for i in range(0,circ.N):
        if the[layer,i]==int(1):
            positions.append(int(i))
        else:
            continue
        
    if positions!=[]:
        circ.measure(*positions)

        
        
    return circ

In [7]:
def create_circuit(N_QUBITS,DEPTH,the):
    circ = pc.circuit.Circuit(N_QUBITS)
    for i in range(DEPTH):
        circ = one_layer_brick_wall(circ,even=True)
        #circ = one_layer_measurement(circ,the,int(2*DEPTH-1-2*i))
        circ = one_layer_brick_wall(circ,even=False)
        #circ = one_layer_measurement(circ,the,int(2*DEPTH-1-2*i-1))
    return circ

In [8]:
def averaged_EE(state_final):
    EE_positions=[]
    for i in range(0,N_QUBITS):
        EE_positions.append([int(i),int(i+1)%N_QUBITS])
        
    EE_list=[]
    for i in range(0,len(EE_positions)):
        EE_list.append(state_final.entropy(EE_positions[i]))
    
    averaged_EE=statistics.mean(EE_list)
        
    return averaged_EE, EE_positions, EE_list

In [9]:
N_QUBITS=10
DEPTH=5
the = theta(N_QUBITS,2*DEPTH)
the
circ=create_circuit(N_QUBITS,DEPTH,the)
circ

array([[1, 1, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
       [1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 1, 1, 0, 0, 0, 0, 1]])

CliffordCircuit(
  |[1,2][3,4][5,6][7,8][9,0]|
  |[0,1][2,3][4,5][6,7][8,9]|
  |[1,2][3,4][5,6][7,8][9,0]|
  |[0,1][2,3][4,5][6,7][8,9]|
  |[1,2][3,4][5,6][7,8][9,0]|
  |[0,1][2,3][4,5][6,7][8,9]|
  |[1,2][3,4][5,6][7,8][9,0]|
  |[0,1][2,3][4,5][6,7][8,9]|
  |[1,2][3,4][5,6][7,8][9,0]|
  |[0,1][2,3][4,5][6,7][8,9]|)
 Unitary:True

In [19]:
stateinitial = stateinitial = pc.stabilizer.zero_state(N_QUBITS)  #Initial stabilizer state
stateinitial

StabilizerState(
   +ZIIIIIIIII
   +IZIIIIIIII
   +IIZIIIIIII
   +IIIZIIIIII
   +IIIIZIIIII
   +IIIIIZIIII
   +IIIIIIZIII
   +IIIIIIIZII
   +IIIIIIIIZI
   +IIIIIIIIIZ)

In [20]:
statefinal=circ.forward(stateinitial) #Final stabilizer state after the circuit applied
statefinal

StabilizerState(
   +IXYZYYXZIZ
   -YXZXZYXIXY
   -ZZYIZIZYXI
   +XXIXYYZZXZ
   -IXZXIYIIII
   -XIIYIXYYII
   -YZXYZZZXZY
   -IXYYXIIIIZ
   +IYYIIIXZYX
   -IXIIIIYYIZ)

In [13]:
#state_final.entropy([0,1]) #Calculate the entanglement entropy of the final stabilizer states

In [21]:
a,b,c= averaged_EE(state_final)
a

2